# Tensorflow 环境搭建及测试

测试安装

In [2]:
import tensorflow as tf
import numpy as np
tf.__version__

'1.11.0'

## 定义参数，输入和输出节点

In [21]:
batch_size = 8
w1= tf.Variable(tf.random_normal([100, 100], stddev=1, seed=1))
w2= tf.Variable(tf.random_normal([100, 100], stddev=1, seed=1))
x = tf.placeholder(tf.float32, shape=(None, 100), name="x-input")
y_= tf.placeholder(tf.float32, shape=(None, 1), name='y-input')
w1, w2, x, y_

(<tf.Variable 'Variable_8:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'Variable_9:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Tensor 'x-input_4:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'y-input_4:0' shape=(?, 1) dtype=float32>)

## 定义前向传播过程，损失函数和反向传播算法

In [22]:
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0))
                                + (1 - y_) * tf.log(tf.clip_by_value(1 - y, 1e-10, 1.0)))
learning_rate = 0.001
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

## 生成模拟数据集

In [23]:
rdm = np.random.RandomState(1)
X = rdm.rand(128,100)
Y = [[int(sum(x) < 50)] for x in X]

## 创建会话运行 tensorflow 程序

In [24]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    # 输出目前（未经训练）的参数取值。
    print(sess.run(w1))
    print(sess.run(w2))
    print("\n")
    
    # 训练模型。
    STEPS = 5000
    for i in range(STEPS):
        start = (i*batch_size) % 128
        end = (i*batch_size) % 128 + batch_size
        sess.run([train_step, y, y_], feed_dict={x: X[start:end], y_: Y[start:end]})
        if i % 1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x: X, y_: Y})
            print("After %d training step(s), cross entropy on all data is %g" % (i, total_cross_entropy))
    
    # 输出训练后的参数取值。
    print("\n")
    print(sess.run(w1))
    print(sess.run(w2))

[[-0.8113182   1.4845988   0.06532937 ...  0.36240223 -2.7669072
   1.9807922 ]
 [ 0.15743099  0.5263604  -2.2120235  ... -1.8342832   0.631131
  -0.24619082]
 [ 1.0542018  -0.75265557 -0.6515841  ...  1.7162865  -0.09378374
   0.33990505]
 ...
 [ 0.02624949 -1.0495203  -0.63945365 ... -0.9287379   1.2584059
  -1.685026  ]
 [ 0.676028    0.00873612  0.4046848  ...  0.99804413  0.6139806
   0.28644598]
 [-0.04969252 -0.45393836 -0.37356263 ...  0.77633905  1.5767121
  -0.28991485]]
[[-0.8113182   1.4845988   0.06532937 ...  0.36240223 -2.7669072
   1.9807922 ]
 [ 0.15743099  0.5263604  -2.2120235  ... -1.8342832   0.631131
  -0.24619082]
 [ 1.0542018  -0.75265557 -0.6515841  ...  1.7162865  -0.09378374
   0.33990505]
 ...
 [ 0.02624949 -1.0495203  -0.63945365 ... -0.9287379   1.2584059
  -1.685026  ]
 [ 0.676028    0.00873612  0.4046848  ...  0.99804413  0.6139806
   0.28644598]
 [-0.04969252 -0.45393836 -0.37356263 ...  0.77633905  1.5767121
  -0.28991485]]


After 0 training step(s), 